In [1]:
import sys
sys.path.insert(0, r"C:\Users\BMSCE CSE.DESKTOP-IUB6THA\Downloads\kshitij\UniEval")

from utils import convert_to_json
from metric.evaluator import get_evaluator

task = 'fact'

evaluator = get_evaluator(task)

C:\Users\BMSCE CSE.DESKTOP-IUB6THA\.conda\envs\kshitij\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\BMSCE CSE.DESKTOP-IUB6THA\.conda\envs\kshitij\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\BMSCE CSE.DESKTOP-IUB6THA\.conda\envs\kshitij\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
from prettytable import PrettyTable

def convert_to_json(output_list, src_list=None, ref_list=None, context_list=None, \
            scores=None, doc_id=None, system_id=None):
    """
        Convert the data into the json format.

        output_list: a list of model output
        src_list: source input for different NLG tasks. For example, source document for summarization
                  and dialogue history for dialogue response generation
        ref_list: human-annotated groundtruth
        context_list: the context needed to evaluate several specific dimension. For example,
                      additional factual information when evaluating engagingness and groundedness in dialogues
        scores: human scores for evaluating the model output. They can be used to calculate the correlation
                between evaluators and human judgements. The scores should be stored in a dictionary. For example,
                {'fluency': 2.0, 'coherence': 3.0} could be the human score for a sample.
        doc_id: the index of the input source. It can be used to calculate summary-level correlation for summarzation
        system_id: the index of the generation system. It can be used to calculate system-level correlation.
    """
    json_data = []
    for i in range(len(output_list)):
        cur = {}
        cur['system_output'] = output_list[i]
        if src_list is not None:
            cur['source'] = src_list[i]
        if ref_list is not None:
            cur['reference'] = ref_list[i]
        if context_list is not None:
            cur['context'] = context_list[i]
        if scores is not None:
            cur['scores'] = scores[i]
        if doc_id is not None:
            cur['doc_id'] = doc_id[i]
        if system_id is not None:
            cur['system_id'] = system_id[i]
        json_data.append(cur)
    return json_data


def add_question(dimension, output, src=None, ref=None, context=None, task=None):
    """
        Add questions to generate input in Bool-QA format for UniEval.

        dimension: specific dimension to be evaluated
        src: source input for different NLG tasks. For example, source document for summarization
             and dialogue history for dialogue response generation.
        output: output text generated by the models
        ref: human-annotataed groundtruth
        context: the context needed to evaluate several specific dimension. For example,
                 additional factual information when evaluating engagingness and groundedness in dialogues.
    """

    input_with_question = []
    for i in range(len(output)):
        # For summarization
        if task == 'summarization':
            if dimension == 'fluency':
                cur_input = 'question: Is this a fluent paragraph? </s> paragraph: ' + output[i]
            elif dimension == 'coherence':
                cur_input = 'question: Is this a coherent summary to the document? </s> summary: ' + output[i] + ' </s> document: ' + src[i]
            elif dimension == 'consistency':
                cur_input = 'question: Is this claim consistent with the document? </s> claim: ' + output[i] + ' </s> document: ' + src[i]
            elif dimension == 'relevance':
                cur_input = 'question: Is this summary relevant to the reference? </s> summary: ' + output[i] + ' </s> reference: ' + ref[i]
            else:
                raise NotImplementedError('The input format for this dimension is still undefined. Please customize it first.')
        # For dialogues
        elif task == 'dialogue':
            if dimension == 'naturalness':
                cur_input = 'question: Is this a natural response in the dialogue? </s> response: ' + output[i]
            elif dimension == 'coherence':
                cur_input = 'question: Is this a coherent response given the dialogue history? </s> response: '\
                            + output[i] + ' </s> dialogue history: ' + src[i]
            elif dimension == 'engagingness':
                cur_input = 'question: Is this an engaging and informative response according to the dialogue history and fact? </s> response: '\
                            + output[i] + ' </s> dialogue history: ' + src[i] + ' </s> fact: ' + context[i]
            elif dimension == 'groundedness':
                cur_input = 'question: Is this response consistent with knowledge in the fact? </s> response: '\
                            + output[i] + ' </s> fact: ' + context[i]
            elif dimension == 'understandability':
                cur_input = 'question: Is this an understandable response in the dialogue? </s> response: ' + output[i]
            else:
                raise NotImplementedError('The input format for this dimension is still undefined. Please customize it first.')
        # For data-to-text
        elif task == 'data2text':
            if dimension == 'naturalness':
                cur_input = 'question: Is this a fluent utterance? </s> utterance: ' + output[i]
            elif dimension == 'informativeness':
                cur_input = 'question: Is this sentence informative according to the reference? </s> sentence: '\
                            + output[i] + ' </s> reference: ' + ref[i]
            else:
                raise NotImplementedError('The input format for this dimension is still undefined. Please customize it first.')
        # For factual consistency detection
        elif task == 'fact':
            if dimension == 'consistency':
                cur_input = 'question: Is this claim consistent with the document? </s> claim: ' + output[i] + ' </s> document: ' + src[i]
            else:
                raise NotImplementedError('No other dimensions for the factual consistency detection task.')
        # For new customized tasks
        else:
            raise NotImplementedError('Other tasks are not implemented, please customize specific tasks here.')
        input_with_question.append(cur_input)
    return input_with_question


def print_scores(scores):
    table = PrettyTable(['Dimensions','Score'])
    print('\nEvaluation scores are shown below:')
    dims = list(scores[0].keys())
    for dim in dims:
        cur_score = 0
        for i in range(len(scores)):
            cur_score += scores[i][dim]
        table.add_row([dim, round(cur_score / len(scores), 6)])
    print(table)

In [3]:
import numpy as np
from nltk import sent_tokenize
from scorer import UniEvaluator  # Make sure this import works after placing scorer.py in the same directory

def evaluate(data, dims=None, overall=True, print_result=False, model_name_or_path="t5-small", task='summarization', device='cuda:0', individual=True):
    """
    Get the scores of all the given dimensions (fluency, consistency, coherence, relevance)

    data: A list of dictionaries, where each dictionary contains:
          - 'source': The original text
          - 'system_output': The generated system output (summary)
          - 'reference' (optional): Reference summary for relevance evaluation

    dims: A list of dimensions to be evaluated. If dims is None, it evaluates four default dimensions:
          coherence, consistency, fluency, relevance.

    overall: Boolean to indicate whether the overall score is calculated as the average of all dimensions.

    print_result: Boolean to print the results on the screen.

    model_name_or_path: The model name or path to use for evaluation, e.g., 't5-small'

    task: The task type (used in scoring if needed, like summarization or other NLP tasks).

    device: The device to use for evaluation ('cpu' or 'cuda:0').
    """

    # Instantiate the scorer
    scorer = UniEvaluator(model_name_or_path=model_name_or_path, device=device)

    n_data = len(data)
    eval_scores = [{} for _ in range(n_data)]

    # Default dimensions if not provided
    if dims is None:
        dims = ['coherence', 'consistency', 'fluency', 'factual consistency']   #add relevance

    for dim in dims:
        print(f'Evaluating {dim} of {n_data} samples !!!')

        if dim == 'consistency' or dim == 'fluency':
            # Sentence-level scores for consistency and fluency
            src_list, output_list = [], []
            n_sents = []  # number of sentences in each summary

            for i in range(n_data):
                if dim == 'consistency':
                    source = data[i]['source']
                else:
                    source = ''
                system_outputs = sent_tokenize(data[i]['system_output'])
                n_sents.append(len(system_outputs))
                for j in range(len(system_outputs)):
                    src_list.append(source)
                    output_list.append(system_outputs[j])

            input_list = add_question(dimension=dim, output=output_list, src=src_list, task=task)
            sent_score = scorer.score(input_list)

            # Calculate average sentence-level scores for each sample
            start_idx = 0
            score = []
            for cur_n_sent in n_sents:
                score.append(sum(sent_score[start_idx:start_idx + cur_n_sent]) / cur_n_sent)
                start_idx += cur_n_sent

        elif dim == 'coherence' or dim == 'relevance':
            # Summary-level scores for coherence and relevance
            src_list, output_list, ref_list = [], [], []

            for i in range(n_data):
                src_list.append(data[i]['source'])
                output_list.append(data[i]['system_output'])
                if dim == 'relevance':
                    ref_list.append(data[i]['reference'])

            input_list = add_question(dimension=dim, output=output_list, src=src_list, ref=ref_list, task=task)
            score = scorer.score(input_list)

        elif dim == 'factual consistency':
            output_list, src_list = [], []

            for i in range(n_data):
                src_list.append(data[i]['source'])
                output_list.append(data[i]['system_output'])

            data = convert_to_json(output_list=output_list, src_list=src_list)
            eval_score = evaluator.evaluate(data)
            score = []

            for i in eval_score:
                temp = i['consistency']
                score.append(temp)

        else:
            raise NotImplementedError(f"The input format for the dimension '{dim}' is still undefined. Please customize it.")

        # Store the scores for the current dimension
        for i in range(n_data):
            eval_scores[i][dim] = score[i]

    # Calculate overall score (average of all evaluated dimensions)
    if overall:
        for i in range(n_data):
            eval_scores[i]['overall'] = np.mean([eval_scores[i][dim] for dim in dims])

    # Print the result if requested
    if print_result:
        print_scores(eval_scores)

    if individual:
        individual_scores = []
        for i in range(n_data):
            temp = [eval_scores[i][dim] for dim in dims]
            individual_scores.append(temp)

        return np.array(individual_scores)

    # Calculate average score across all the dimensions except 'overall'
    avg_score = []
    for i in range(n_data):
        # Exclude 'overall' from the averaging
        dimensions = [dim for dim in dims if dim != 'overall']
        avg_score.append(np.mean([eval_scores[i][dim] for dim in dimensions]))

    return avg_score

In [4]:
# Import all required libraries
import torch
import transformers
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, TrainingArguments
from trl import RewardTrainer, PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model
from datasets import Dataset
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training, TaskType
import bitsandbytes as bnb
import numpy as np
from tqdm import tqdm
import time
import sys

# Add UniEval to path and import
sys.path.append(r"C:\Users\BMSCE CSE.DESKTOP-IUB6THA\Downloads\kshitij\UniEval")
from utils import convert_to_json
from metric.evaluator import get_evaluator

# Configuration
DATA_PATH = "combined_clinical_notes.csv"
# MODEL_PATH = r"D:\kshitij-weights-folder\qwen-aloe-9-4-base-fine-tune"
MODEL_PATH = "gpt2" 
# PEFT_ADAPTER_PATH = r"D:\kshitij-weights-folder\qwen-aloe-9-4-base-fine-tune-peft-adapaters"
MEDICAL_PROMPT = "\nGenerate a concise medical summary focusing on key findings and treatment plans:"

# Load and prepare data
df = pd.read_csv(DATA_PATH)
train_df, temp_df = train_test_split(df, test_size=0.4, random_state=42)
eval_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
dataset = Dataset.from_pandas(eval_df.rename(columns={"dialogue": "review"}))

# Tokenizer setup
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, padding_side='left')
tokenizer.pad_token = tokenizer.eos_token

# Dataset preprocessing
def preprocess_function(examples):
    return {
        "input_ids": tokenizer.encode(examples["review"], truncation=True, padding="max_length", max_length=512),
        "query": tokenizer.decode(tokenizer.encode(examples["review"], truncation=True, padding="max_length", max_length=512), skip_special_tokens=True)
    }

dataset = dataset.map(preprocess_function, batched=False)
dataset.set_format("pytorch")

# Model configuration
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    quantization_config=bnb_config,
    device_map="auto"
)
base_model = prepare_model_for_kbit_training(base_model)

# PEFT/LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["c_attn", "c_proj"],
)

# model_with_lora = get_peft_model(base_model, lora_config)
model = AutoModelForCausalLMWithValueHead.from_pretrained(base_model, peft_config=lora_config).to("cuda")

# Reference model
ref_model = create_reference_model(model).to("cuda")
ref_model.eval()
for param in ref_model.parameters():
    param.requires_grad = False

NUM_CANDIDATES = 2

# PPO Configuration
ppo_config = PPOConfig(
    model_name=MODEL_PATH,
    ppo_epochs=1,
    gradient_accumulation_steps=1,
    steps=5,
    batch_size=1*NUM_CANDIDATES,
    mini_batch_size=1*NUM_CANDIDATES,
    learning_rate=2e-5,
    log_with=None,
    # project_kwargs={"logging_dir": r"D:\kshitij-weights-folder\gpt2-rl-logs"}
)

# optimizer = torch.optim.AdamW(
#     filter(lambda p: p.requires_grad, ppo_model.parameters()),
#     lr=2e-5,
#     eps=1e-5,  # Slightly larger epsilon for stability
# )

# Initialize PPO Trainer
ppo_trainer = PPOTrainer(
    config=ppo_config,
    model=model,
    ref_model=ref_model,
    tokenizer=tokenizer,
    dataset=dataset,
    optimizer=bnb.optim.Adam8bit(model.parameters(), lr=ppo_config.learning_rate)
)

# Evaluation setup
# sum_eval = get_evaluator("summarization", "cuda"="cuda" if torch.cuda.is_available() else "cpu")

def get_score(src, res):
    # weights = np.array([0.1, 0.2, 0.3, 0.4])  # coherence, consistency, fluency, factual consistency
    # sample_data = [{"source": q, "system_output": r} for q, r in zip(game_data["query"], game_data["response"])]
    
    # scores = sum_eval.evaluate(sample_data, overall=False)
    # scores = evaluate(sample_data, overall=False)
    # weighted_scores = []
    
    # for dimension_scores in scores:
    #     adjusted = np.where(
    #         dimension_scores < 0.5,
    #         -dimension_scores * weights,
    #         dimension_scores * weights
    #     )
    #     weighted_scores.append(torch.tensor(np.sum(adjusted)/4, dtype=torch.float32).to(model.pretrained_model.device))
    
    # return weighted_scores

    data = convert_to_json(
        output_list=res,
        src_list=src,
    )
    # raw = sum_eval.evaluate(data, print_result=True)
    # dims = ['coherence', 'consistency', 'fluency', 'factual consistency']
    raw = evaluate(data, overall=False)
    score = [
        [d[0], d[1], d[2], d[3]]
        for d in raw
    ]
    scores = torch.tensor(score, dtype=torch.float32).numpy()  # CPU (B,4

    k = len(res)
    dom_counts = np.zeros(k)
    
    for i in range(k):
        for j in range(k):
            if i == j:
                continue
            # Check dominance: i dominates j if all scores are >= and at least one is >
            if np.all(scores[i] >= scores[j]) and np.any(scores[i] > scores[j]):
                dom_counts[i] += 1
    
    # Convert to [-1, 1] range reward
    max_dom = k - 1
    if max_dom > 0:
        rewards = 2 * (dom_counts / max_dom) - 1
    else:
        rewards = np.zeros(k)
    
    return rewards


max_position_embeddings = model.pretrained_model.config.max_position_embeddings 
# Training loop
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    # "max_new_tokens": 64,  # Increased for better summary generation
    "eos_token_id": -1,
    "max_length": max_position_embeddings,
    "max_new_tokens": 64
}
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Use consistent compute dtype
COMPUTE_DTYPE = torch.float32  # Using float32 to avoid dtype issues

for epoch in range(1):
    for batch_idx, batch in enumerate(tqdm(ppo_trainer.dataloader)):
        (logs, game_data,) = (
            dict(),
            dict(),
        )

        # task_list = choices(ctrl_str, k=config.batch_size)
        # game_data["query"] = [t + q for t, q in zip(task_list, batch["query"])]
        game_data["query"] = [q for q in batch["query"]]
        # query_tensors = [torch.cat((ctrl_tokens[t], input_ids)) for t, input_ids in zip(task_list, batch["input_ids"])]
        query_tensors = [input_ids for input_ids in batch["input_ids"]]
        all_outs = []
        
        response_tensors = []
        for query in query_tensors:
            for _ in range(NUM_CANDIDATES):
                original_notes = tokenizer.decode(query)
                
                # Combine with medical prompt only during generation
                full_prompt = f"{MEDICAL_PROMPT}{original_notes}" #TRy with full prompt here
                full_prompt_tensor = tokenizer.encode(full_prompt, return_tensors="pt").to("cuda").squeeze(0)
                
                with torch.no_grad():
                    response = ppo_trainer.generate(
                        full_prompt_tensor,
                        **generation_kwargs
                    )
                # Ensure response doesn't exceed max length
                response = response[:, :generation_kwargs["max_new_tokens"]]
                all_outs.append(response)
                response_tensors.append(response.squeeze())

            outs = torch.stack(all_outs, dim=1)
            B, K, _ = outs.shape
            
            # Decode outputs for evaluation
            hyps = []
            for b in range(B):
                hyps_b = []
                for k in range(K):
                    try:
                        text = tokenizer.decode(outs[b, k], skip_special_tokens=True)
                        hyps_b.append(text)
                    except Exception as e:
                        print(f"Error decoding text: {e}")
                        hyps_b.append("")  # Add empty string as fallback
                hyps.append(hyps_b)

            # rewards = []
            # for b in range(len(batch['input_ids'])):
            #     # Get scores for all candidates (K, 4)
            #     scores = get_score(
            #         batch['query'][b] * NUM_CANDIDATES,
            #         hyps[b]
            #     ).numpy()

            #     print("check")

            #     dom_counts = np.zeros(NUM_CANDIDATES)
            #     for i in range(NUM_CANDIDATES):
            #         for j in range(NUM_CANDIDATES):
            #             if i == j:
            #                 continue
            #             # Check if i dominates j
            #             if np.all(scores[i] >= scores[j]) and np.any(scores[i] > scores[j]):
            #                 dom_counts[i] += 1

            #     max_dom = NUM_CANDIDATES - 1
            #     scalar_rewards = 2 * (dom_counts / max_dom) - 1
            #     rewards.append(scalar_rewards)

            # flat_queries = []
            # flat_responses = []
            # flat_rewards = []

            # for b in range(len(batch['input_ids'])):
            #     for k in range(NUM_CANDIDATES):
            #         flat_queries.append(batch['input_ids'][b])
            #         flat_responses.append(outs[b, k])
            #         flat_rewards.append(torch.tensor([rewards[b][k]], device="cuda"))

            # stats = ppo_trainer.step(
            #     queries   = flat_queries,    # e.g. [ q0, q0 ]
            #     responses = flat_responses,  # e.g. [ r0, r1 ]
            #     scores    = flat_rewards     # e.g. [ s0, s1 ]
            # )

            flat_queries, flat_responses, flat_rewards = [], [], []
                    
            for b in range(B):
                try:
                    # Calculate rewards
                    rewards_b = get_score(
                        [batch['query'][b]] * K, 
                        hyps[b],
                    )
                    
                    # Flatten for PPO
                    for k in range(K):
                        flat_queries.append(batch['input_ids'][b])
                        flat_responses.append(outs[b, k])
                        flat_rewards.append(torch.tensor([rewards_b[k]], device=DEVICE, dtype=COMPUTE_DTYPE))
                        
                except Exception as e:
                    print(f"Error computing rewards: {e}")
                    continue
            
            # Safety check
            if len(flat_queries) != ppo_config.batch_size:
                print(f"Batch size mismatch: expected {ppo_config.batch_size}, got {len(flat_queries)}")
                continue

            try:
                # Verify shapes match
                print(f"Queries: {len(flat_queries)}, Responses: {len(flat_responses)}, Rewards: {len(flat_rewards)}")
                
                # Manual memory management
                # torch.cuda.empty_cache()
                
                # Do PPO step
                stats = ppo_trainer.step(
                    queries=flat_queries,
                    responses=flat_responses,
                    scores=flat_rewards
                )
                
                # Success! Log the output
                print(f"Batch {batch_idx} PPO step successful!")
                print(f"Sample output: {hyps[0][0][:100]}...")
                avg_reward = np.mean([r.item() for r in flat_rewards])
                print(f"Average reward: {avg_reward:.4f}")
                
            except RuntimeError as e:
                print(f"Error in PPO step: {e}")
                
                # If still running into CUDA errors, try moving to CPU
                if "CUDA" in str(e) and DEVICE != "cpu":
                    print("\nContinuing to encounter CUDA errors. Try two options:")
                    print("1. Change DEVICE = 'cpu' at the top of the script")
                    print("2. Or use the non-quantized model version\n")
                    
                # Clear memory and continue
                # if torch.cuda.is_available():
                #     torch.cuda.empty_cache()
            
            # if batch_idx % 10 == 0:
            #     print(f"Epoch {epoch+1}, Batch {batch_idx}")
            #     print(f"Sample output: {hyps[0][0][:100]}...")
            #     print(f"Average reward: {np.mean([r.item() for r in flat_rewards]):.4f}")

    print(f"✅ Epoch {epoch+1}/3 complete")
    
print("🎉 PPO fine-tuning done")
#         print(response_tensors)
        # game_data["response"] = [tokenizer.decode(r) for r in response_tensors]

        # print("check")

        # texts = [q + r for q, r in zip(batch["query"], game_data["response"])]
        # logits = get_score(game_data)
        # rewards = logits
        # rewards = pos_logit_to_reward(logits, task_list)
        # rewards = [torch.tensor([1.0], device=query_tensors[0].device) for _ in range(len(texts))]

        #### Run PPO training
        # t = time.time()
        # stats = ppo_trainer.step(query_tensors, response_tensors, rewards)


C:\Users\BMSCE CSE.DESKTOP-IUB6THA\.conda\envs\kshitij\Lib\site-packages\trl\trainer\ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(
C:\Users\BMSCE CSE.DESKTOP-IUB6THA\.conda\envs\kshitij\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
  0%|                                                                                 | 0/46 [00:00<?, ?it/s]Both `max_new_tokens` (=64) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=64) and `max_length`(=1024) seem to have been set. `max_new_to

Evaluating coherence of 2 samples !!!



100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  8.92it/s]


Evaluating consistency of 2 samples !!!



100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 11.71it/s]


...

Evaluating fluency of 4 samples !!!



100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 65.76it/s]


Evaluating factual consistency of 4 samples !!!
Evaluating consistency of 4 samples !!!



100%|████████████████████████████████████████████████████████████████████████| 46/46 [07:57<00:00, 10.39s/it]

Batch size mismatch: expected 2, got 4
✅ Epoch 1/3 complete
🎉 PPO fine-tuning done


In [5]:
ppo_trainer.model.pretrained_model.save_pretrained("D:\kshitij-weights-folder\gpt-2-tuned-ppo")
tokenizer.save_pretrained("D:\kshitij-weights-folder\gpt-2-tuned-ppo")


('D:\\kshitij-weights-folder\\gpt-2-tuned-ppo\\tokenizer_config.json',
 'D:\\kshitij-weights-folder\\gpt-2-tuned-ppo\\special_tokens_map.json',
 'D:\\kshitij-weights-folder\\gpt-2-tuned-ppo\\vocab.json',
 'D:\\kshitij-weights-folder\\gpt-2-tuned-ppo\\merges.txt',
 'D:\\kshitij-weights-folder\\gpt-2-tuned-ppo\\added_tokens.json',
 'D:\\kshitij-weights-folder\\gpt-2-tuned-ppo\\tokenizer.json')